In [1]:
#!pip install instaloader

In [2]:
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import pandas as pd

L = instaloader.Instaloader()

In [3]:
with open('../login_info.txt', encoding='utf-8') as file:
    content = file.read()
    username = content.split()[0]
    pswd = content.split()[1]

# L.login(username, pswd)
# L.save_session_to_file('infonsociol_session_file')

In [4]:
username

'infonsociol'

In [5]:
pswd

'Bakamono?!0'

In [6]:
account_usernames = ["NBCNews", 
                     "FoxNews", 
                     "CNN", 
                     "NYTimes",
                     "WashTimes", 
                     "WSJ", 
                     "BBCNews", 
                     "Reuters",
                     "Time", 
                     "Aljazeeraenglish", 
                     "Tabletmag", 
                     "voxdotcom",
                     "vicenews", 
                     "Axios", 
                     "HuffPost"]

For details of properties of Post class:\
https://instaloader.github.io/module/structures.html#posts

For details of properties of Profile class:\
https://instaloader.github.io/module/structures.html?highlight=postcomment#profiles


In [7]:
# summary info of each account's posts
# should have same length as account_usernames
posts_df_list = []


# lists to generate summary info of basic account info
# should have same length as account_usernames
userid_list = []
followers_list = []
full_name_list = []
is_verified_list = []
post_count_list = []    # counts of posts since 231001

for username in account_usernames:
    posts = instaloader.Profile.from_username(L.context, username).get_posts()

    SINCE = datetime(2023, 10, 1)
    UNTIL = datetime.today()
    
    url_code_list = []
    time_utc_list = []
    content_types_list = []
    captions_list = []
    like_counts_list = []
    comment_counts_list = []

    for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
    
        # details of post: date&time, count of likes, count of comments, url
        url_code_list.append(post.shortcode)
        time_utc_list.append(post.date_utc)
        content_types_list.append(post.typename)
        captions_list.append(post.caption)
        like_counts_list.append(post.likes)
        comment_counts_list.append(post.comments)
        # L.download_post(post, username)
    
    # create df with info of each posts from this account
    df_name = "df_" + username
    exec_string = """{} = pd.DataFrame(data={{
    'url_code': url_code_list, 
    'time_utc': pd.to_datetime(time_utc_list), 
    'type': content_types_list,
    'caption': captions_list,
    'likes': like_counts_list, 
    'comments': comment_counts_list}})
    """
    exec(exec_string.format(df_name))
    
    # export df to csv
    exec_string = "{}.to_csv()"
    exec(exec_string.format(df_name))
    
    # add the dataframe to the list df_list
    exec_string = "posts_df_list.append({})"
    exec(exec_string.format(df_name))
    
    print("Done with posts of", username)
    print("{} has {} posts since October 1st.".format(username, len(url_code_list)))
    post_count_list.append(len(url_code_list))
    
    
    # append this account info
    userid_list.append(post.owner_profile.userid)
    followers_list.append(post.owner_profile.followers)
    full_name_list.append(post.owner_profile.full_name)
    is_verified_list.append(post.owner_profile.is_verified)
    
d = {'user_id': userid_list, 'followers': followers_list, 'full_name':full_name_list, 'is_verified':is_verified_list, 'posts_since_231001': post_count_list}
df_accounts = pd.DataFrame(data=d)

JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]
JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]


ConnectionException: JSON Query to graphql/query: HTTP error code 401.

In [ ]:
df_NBCNews.shape[0], df_FoxNews.shape[0], df_CNN.shape[0], df_NYTimes.shape[0], df_WashTimes.shape[0]

In [ ]:
df_WSJ.shape[0],df_BBCNews.shape[0],df_Reuters.shape[0],df_Time.shape[0],df_Aljazeeraenglish.shape[0]

In [ ]:
df_Tabletmag.shape[0], df_voxdotcom.shape[0],df_vicenews.shape[0],df_Axios.shape[0],df_HuffPost.shape[0]

In [ ]:
post.owner_profile.followers